<a href="https://colab.research.google.com/github/MahlerTom/Siamese-Neural-Networks/blob/master/EX02_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Statistics


# Implement Siamese Network

## Changes to the model:

1. In the original paper, a dense layer with size of 4096 was used. However, in our case, we have to decrease the size of the such layer so that we won't have the probelm of an oversized tensor. Therefore we used a dense layer with size of 2048.
2. 